In [2]:
import pandas as pd
from datetime import datetime, timedelta, date
from collections import defaultdict
import json

In [3]:
df = pd.read_csv('./data/2020_jan.csv')

In [4]:
new_times_lap = []
for index, time in df['Lap Time'].iteritems():
    if time.count(':') == 1:
        new_times_lap.append(datetime.strptime(time, '%M:%S.%f'))
    else:
        new_times_lap.append(datetime.strptime(time, '%H:%M:%S.%f'))
df['Lap Times'] = new_times_lap

In [5]:
new_times_s = []
date_index = 100000
for index, time in df['Session Time'].iteritems():
    if time.startswith('24') and time.count(':') > 1:
        date_index = index
        new_time = '01:00' + time[2:]
        new_date = datetime.strptime(new_time, '%d:%H:%M:%S.%f')
        new_date_plus = new_date + timedelta(days=1)
        new_times_s.append(new_date_plus)
    elif index > date_index:
        new_time = '01:00:0' + time
        new_date = datetime.strptime(new_time, '%d:%H:%M:%S.%f')
        new_date_plus = new_date + timedelta(days=1)
        new_times_s.append(new_date_plus)
    elif time.count(':') == 1:
        new_times_s.append(datetime.strptime(time, '%M:%S.%f'))
    else:
        new_times_s.append(datetime.strptime(time, '%H:%M:%S.%f'))
df['Session Times'] = new_times_s

In [6]:
df['Prev Session Time'] = df['Session Times'].subtract(df['Lap Times'])

In [7]:
for i in range(1,14):
    num_str = str(i)
    if i < 10: num_str = '0' + num_str
    num_str = 'S' + num_str
    
    vals = []
    for ind, split_time in df[num_str].iteritems():
        vals.append(timedelta(seconds=split_time))
    df[num_str] = vals
    totals = df[num_str] + df['Prev Session Time']
    
    for j in range(i-1, 0, -1):
        new_num_str = str(j)
        if j < 10: new_num_str = '0' + new_num_str
        new_num_str = 'S' + new_num_str
        totals += df[new_num_str]
    df[num_str+'totals'] = totals

In [20]:
section = defaultdict(lambda: defaultdict(list))
for index, row in df.iterrows():
    section[row['Driver']]['S01'].append((row['Prev Session Time'].total_seconds(), row['S01totals'].total_seconds()))
    section[row['Driver']]['S02'].append((row['S01totals'].total_seconds(), row['S02totals'].total_seconds()))
    section[row['Driver']]['S03'].append((row['S02totals'].total_seconds(), row['S03totals'].total_seconds()))
    section[row['Driver']]['S04'].append((row['S03totals'].total_seconds(), row['S04totals'].total_seconds()))
    section[row['Driver']]['S05'].append((row['S04totals'].total_seconds(), row['S05totals'].total_seconds()))
    section[row['Driver']]['S06'].append((row['S05totals'].total_seconds(), row['S06totals'].total_seconds()))
    section[row['Driver']]['S07'].append((row['S06totals'].total_seconds(), row['S07totals'].total_seconds()))
    section[row['Driver']]['S08'].append((row['S07totals'].total_seconds(), row['S08totals'].total_seconds()))
    section[row['Driver']]['S09'].append((row['S08totals'].total_seconds(), row['S09totals'].total_seconds()))
    section[row['Driver']]['S10'].append((row['S09totals'].total_seconds(), row['S10totals'].total_seconds()))
    section[row['Driver']]['S11'].append((row['S10totals'].total_seconds(), row['S11totals'].total_seconds()))
    section[row['Driver']]['S12'].append((row['S11totals'].total_seconds(), row['S12totals'].total_seconds()))
    section[row['Driver']]['S13'].append((row['S12totals'].total_seconds(), row['S13totals'].total_seconds()))

In [25]:
with open('section.json', 'w') as outfile:
    json.dump(section, outfile)

In [11]:
section_mod = defaultdict(list)
for index, row in df.iterrows():
    section_mod[row['Driver']].append((1, row['Prev Session Time'].total_seconds(), row['S01totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((2, row['S01totals'].total_seconds(), row['S02totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((3, row['S02totals'].total_seconds(), row['S03totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((4, row['S03totals'].total_seconds(), row['S04totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((5, row['S04totals'].total_seconds(), row['S05totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((6, row['S05totals'].total_seconds(), row['S06totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((7, row['S06totals'].total_seconds(), row['S07totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((8, row['S07totals'].total_seconds(), row['S08totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((9, row['S08totals'].total_seconds(), row['S09totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((10, row['S09totals'].total_seconds(), row['S10totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((11, row['S10totals'].total_seconds(), row['S11totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((12, row['S11totals'].total_seconds(), row['S12totals'].total_seconds(), row.SPI))
    section_mod[row['Driver']].append((13, row['S12totals'].total_seconds(), row['S13totals'].total_seconds(), row.SPI))

In [15]:
MAXTIME = df['S13totals'].max().total_seconds()
def roundTo(x, base=20):
    return base * round(x/base)
MAXTIME = roundTo(MAXTIME, 20)

In [16]:
num_drivers = defaultdict(lambda: defaultdict(list))
for t in range(20, MAXTIME, 20):
    for driver in section_mod:
        for interval in section_mod[driver]:
            if interval[1] <= t <= interval[2]:
                num_drivers[t][interval[0]].append((interval[1], interval[2], interval[3]))
                break


In [46]:
with open('20_count_wspeed.json', 'w') as outfile:
    json.dump(num_drivers, outfile)